In [ ]:
import os
import spacy
import pandas as pd
import psycopg2
import openai
import json
from sqlalchemy import create_engine

# Set environment variables securely for sensitive information
# DATABASE_HOST = "database-test-postgress-instance.cpk2uyae6iza.ap-south-1.rds.amazonaws.com"
# DATABASE_USERNAME = "postgres"
# DATABASE_PASSWORD ="valign#123"  # Make sure this is set in your environment variables
# DATABASE_DB = "zoho_projects_data_v2_backup"
# PORT = 5432

# Set OpenAI API key
OPENAI_API_KEY = ""  # Replace with your OpenAI API key
openai.api_key = OPENAI_API_KEY

# Function to connect to the database
def connect_to_db():
    try:
        connection_url = f"postgresql+psycopg2://{DATABASE_USERNAME}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{PORT}/{DATABASE_DB}"
        engine = create_engine(connection_url)
        return engine
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        raise

# Function to get schema information from the database
def get_schema_info():
    engine = connect_to_db()
    try:
        query = """
        SELECT table_name, column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = 'public';
        """
        df = pd.read_sql_query(query, con=engine)
        return df
    except Exception as e:
        print(f"Error fetching schema information: {e}")
        raise
    finally:
        engine.dispose()

# Function to filter schema based on extracted entities
def get_schema_filter_info(extracted_entities):
    # Extract the table names from the `extracted_entities` dictionary
    tables_of_interest = list(extracted_entities.keys())
    print("tables_of_interest",tables_of_interest)

    engine = connect_to_db()
    try:
        # Query to get schema information for the specified tables
        query = f"""
        SELECT table_name, column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = 'public'
          AND table_name IN ({', '.join([f"'{table}'" for table in tables_of_interest])});
        """
        df = pd.read_sql_query(query, con=engine)
        return df
    except Exception as e:
        print(f"Error fetching schema information: {e}")
        raise
    finally:
        engine.dispose()

# Function to extract features with OpenAI based on user input and schema
def extract_features_with_openai(user_input, processed_schema_df):
    schema_json = processed_schema_df.to_json(orient='records')

    # Define dynamic extraction keywords for aggregation and entity-specific queries
    dynamic_keywords = {
        "aggregation": ["highest", "most", "maximum", "max", "largest", "lowest", "least", "minimum", "min", "count", "sum", "average"],
        "entity_specific": ["who", "where", "which", "what", "how many", "how much"]
    }

    # Prompt preparation: Guide OpenAI to understand schema and extract relevant features
    prompt = f"""
    ### Database Schema Overview
    The following schema provides the columns and tables available in the database. Use this schema to interpret user input accurately:
    {schema_json}

    ### User Query
    The user has input: "{user_input}"

    ### Instructions
    1. **Understand User Intent**:
       - Carefully read the user query both left-to-right and right-to-left to identify potential entities, relationships, and operations.
       - Use a "chain of thoughts" approach to break down the query into parts and analyze the intent for each segment.
       - Identify if the query indicates aggregation (e.g., maximum, minimum, count) or specific entities (e.g., user names, project names, milestone names, status etc.).
       - Ensure that synonyms and closely related terms are mapped accurately to schema components.

    2. **Extract Relevant Entities**:
       - Match user input to text-based columns in the schema (e.g., `varchar`, `char`, `text`).
       - Handle partial matches, synonyms, and spelling variations by considering contextual clues from the schema.
       - Resolve foreign keys to their corresponding descriptive columns (e.g., `assigneeid` -> `username`,`projectid` -> `projectname`,`milestoneid` -> `milestonename`).

    3. **Classify Query Type**:
       - Determine if the query is:
         a. An **aggregation query**, requiring calculations like maximum, minimum, or count.
         b. An **entity-specific query**, focusing on particular items or details.
       - Provide reasoning for the classification in the output.

    4. **Expected Output Format**:
       - A JSON object containing:
         a. `query_type`: Either "aggregation" or "entity_extraction".
         b. `intent_analysis`: A breakdown of user intent and reasoning.
         c. `extracted_entities`: Relevant tables, columns, and matched values from the schema.

    5. **Handle Foreign Keys Dynamically**:
       - If a foreign key column exists (e.g., `assigneeid`,`projectid`,`milestoneid`) but points to a non-text field, exclude it.
       - If the foreign key references a string-based column (e.g., `username`,`projectname`,`milestonename`), include only the string-based column from the related table.

    ### Example Outputs
    For the query "Who logged the highest hours last month?":
    {{
        "query_type": "aggregation",
        "intent_analysis": "The query asks for the user who logged the most hours, requiring aggregation on the hours column.",
        "extracted_entities": {{
            "users_zoho_projects_": {{
                "username": "resolved dynamically"
            }}
        }}
    }}

    For the query "How many hours did Rishi log last month?":
    {{
        "query_type": "entity_extraction",
        "intent_analysis": "The query asks for specific details about a user (Rishi) and the hours logged.",
        "extracted_entities": {{
            "users_zoho_projects_": {{
                "username": "Rishi"
            }}
        }}
    }}

    For the query "What is the status of project Voice enabled which is assigned to Basavaraj?":
    {{
        "query_type": "entity_extraction",
        "intent_analysis": "The query asks for the status of a project assigned to a specific user.",
        "extracted_entities": {{
            "projects_zoho_projects_": {{
                "projectname": "Voice enabled",
                "status": "resolved dynamically"
            }},
            "users_zoho_projects_": {{
                "username": "Basavaraj"
            }}
        }}
    }}

    ### User Query Analysis and Output
    """
    try:
        # Use OpenAI to analyze the query dynamically
        response = openai.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=[
                {"role": "system", "content": "You are an expert assistant skilled in understanding and extracting entities from user queries based on a database schema."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=7000,
            temperature=0.2
        )

        # Extract the response content
        response_content = response.choices[0].message.content.strip()
        print("response_content:", response_content)

        # Parse the JSON response
        extracted_features = json.loads(response_content)
        print("extracted_features:", extracted_features)

        return extracted_features

    except openai.OpenAIError as e:
        print(f"Error with OpenAI API: {e}")
        raise
    except json.JSONDecodeError as e:
        print(f"Error parsing OpenAI response: {e}")
        raise

# Function to generate SQL query using schema and data context
def generate_sql_query(processed_schema_str, response_content):
    # Extract intent and entities from response_content
    intent_analysis = response_content.get("intent_analysis", "")
    extracted_entities = response_content.get("extracted_entities", {})

    # Create a prompt with the context provided by response_content
    prompt = f"""
    You are a PostgreSQL expert. Given an input question, create a syntactically correct PostgreSQL query and return ONLY the generated query. Use the following guidelines for SQL generation:
    ### Database Schema Overview
    The following schema provides the columns and tables available in the database:
    {processed_schema_str}


    ### User Intent Analysis
    {intent_analysis}

    ### Extracted Entities
    The following entities have been extracted from the input:
    {extracted_entities}

    ### Instructions
    - The input may contain partial or similar names for entities, such as project names, user names, or status descriptions. Handle these using `ILIKE` operators with `%` on both sides of the string to allow flexible matching and accurate querying.
    - **Do not use column names directly from user input** without validation. Cross-check user terms against the provided database schema and select the most relevant and existing columns.
    - If the user input mentions a column that doesn't exist (e.g., `due_date`), **substitute it with a semantically similar, valid column** from the schema (e.g., use `end_date` if `due_date` is requested but not found).
    - Ensure you match project names like 'IIFL Samasta', 'IIfl Samasta', 'IIFL', 'iifl smasta', 'Iiflsmsota' (or any spelling variations) to the correct project name from the database.
    - Use the column `instnm` whenever the question involves institute names, and ensure it is correctly associated with `unitid` in the query.
    - When context involves multiple tables, use JOIN operations, ensuring only appropriate columns (e.g., `unitid`) are used for these joins while maintaining referential integrity.
    - **Whenever using foreign key columns**, always **select the corresponding string column from the related table** for clarity. For example, if `owner_id` from the `projects` table is a foreign key referencing `user_id` in the `users` table, select the `user_name` (or relevant name column) from the `users` table, not just the ID.
    - For calculations such as averages or ratios, ensure proper aggregation using `AVG()`, `SUM()`, or other relevant functions, maintaining clarity and correctness in grouped results.
    - Apply detailed filtering criteria from the input in the `WHERE` clause, using logical operators like `AND` and `OR` to combine conditions effectively.
    - Use date and timestamp functions like `TO_TIMESTAMP()` and `EXTRACT` to handle date parsing and extraction of date components as needed.
    - Employ the `GROUP BY` clause when aggregating by categories or attributes for clear grouping.
    - Maintain readability with table and column aliases, especially for complex joins or subqueries.
    - Utilize subqueries or Common Table Expressions (CTEs) for breaking down complex logic into simpler, modular parts for better performance and clarity.
    - Limit the number of rows in the result set to a maximum of 100 rows unless specified otherwise to optimize performance and avoid excessive data output.

    - Ensure comprehensive understanding of user input and intent by interpreting queries from both left to right and right to left to capture full context.
    - Use a range of SQL operators (`=`, `!=`, `LIKE`, `ILIKE`, `IN`) and functions (`IFNULL`, `ISNULL`, `COALESCE`) as needed to handle specific conditions, null handling, and flexible matching.
    - Integrate advanced SQL clauses like `CASE` and `HAVING` when addressing conditional logic or grouped results.
    - Safeguard the handling of NULLs to avoid logical errors, ensuring expressions that involve NULLs behave as expected.
    - Address complex filtering by using nested conditions and logical operators, aligning the SQL with the nuanced requirements of the user query.
    - Confirm all table relationships in JOINs are clear and maintain referential integrity to support correct data linkage.
    - Prioritize performance by avoiding unnecessary complexity or inefficient constructs, focusing on optimized query design.
    - Handle edge cases where user input may imply similar or interchangeable column names, ensuring the selected columns are contextually appropriate (e.g., using `end_date` instead of `due_date` if the former is present in the schema).
    - **For foreign key relationships, always ensure that the related table’s string column (such as user names, project names, etc.) is also selected when referring to the foreign key.**

    Ensure the final SQL query is well-structured, efficient, and accurately reflects the user's request based on the provided input. The response should not include explanations, comments, or any other content beyond the generated SQL query.
    """

    try:
        # Call OpenAI to generate the SQL query
        response = openai.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=[
                {"role": "system", "content": "You are an expert in PostgreSQL query generation."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=7000,
            temperature=0.2
        )

        # Extract generated query from response
        generated_query = response.choices[0].message.content.strip()
        print("Generated SQL Query:", generated_query)

        return generated_query

    except openai.OpenAIError as e:
        print(f"Error with OpenAI API: {e}")
        raise



In [ ]:
# Run the main script
if __name__ == "__main__":
    processed_schema_df = get_schema_info()

    # Sample user input
    user_input = "What are the issues associated with all 'iDatalytics' projects?"

    # Extract features using OpenAI
    response_content = extract_features_with_openai(user_input, processed_schema_df)
    print("Query Type", type(response_content.get("extracted_entities")))
    print("Extractewd Featues Clean", response_content.get("extracted_entities"))

    # Filter schema based on extracted entities
    filtered_schema_df = get_schema_filter_info(response_content.get("extracted_entities", {}))
    #print(f"filtered_schema_df: {filtered_schema_df}")

    # Generate SQL query based on user input and extracted features
    generated_sql_query = generate_sql_query(filtered_schema_df.to_json(orient='records'), response_content)
    print(f"Generated SQL Query: {generated_sql_query}")


response_content: {
    "query_type": "entity_extraction",
    "intent_analysis": "The query seeks to identify issues related to projects specifically named 'iDatalytics'. This involves extracting issues associated with a particular project name.",
    "extracted_entities": {
        "projects_zoho_projects_": {
            "projectname": "iDatalytics"
        },
        "issues_zoho_projects_": {
            "projectid": "resolved dynamically"
        }
    }
}
extracted_features: {'query_type': 'entity_extraction', 'intent_analysis': "The query seeks to identify issues related to projects specifically named 'iDatalytics'. This involves extracting issues associated with a particular project name.", 'extracted_entities': {'projects_zoho_projects_': {'projectname': 'iDatalytics'}, 'issues_zoho_projects_': {'projectid': 'resolved dynamically'}}}
Query Type <class 'dict'>
Extractewd Featues Clean {'projects_zoho_projects_': {'projectname': 'iDatalytics'}, 'issues_zoho_projects_': {'projec